In [2]:
# 8-2 Proc#new以外のProcオブジェクトの作り方

In [3]:
lambda_obj = lambda { 1 }

#<Proc:0x0000561034409e18@(pry):3 (lambda)>

In [4]:
lambda_obj.class

Proc

In [5]:
lambda_obj.call

1

In [6]:
# 8-2-2 Kernel.#lambdaメソッド

In [7]:
inc = ->(x) { x + 1 }

#<Proc:0x0000561034b90a38@(pry):7 (lambda)>

In [8]:
inc.(1)

2

In [9]:
lo = lambda {|x| x + 1}

#<Proc:0x0000561034c0ae50@(pry):9 (lambda)>

In [10]:
lo.call(10)

11

In [11]:
# 8-3 Proc.new/Kernel.proc/Kernel.lambda

In [13]:
def proc_return
  Proc.new {return 1; puts :unreachable}.call
  :unreachable
end

:proc_return

In [14]:
proc_return

1

In [15]:
Proc.new {return 1; puts :unreachable}.call

LocalJumpError: unexpected return

In [16]:
lambda { return 1; puts :unreachable}.call

1

In [17]:
def lambda_return
  lambda {return 1; puts :unreachable}.call
  :reachable
end

lambda_return

:reachable

In [18]:
Proc.new {break 1; puts :unreachable}.call

LocalJumpError: break from proc-closure

In [19]:
def proc_break
  Proc.new { break 1 ; puts :unreachable}.call
end

proc_break

LocalJumpError: break from proc-closure

In [20]:
lambda { break 1; puts :unreachable }.call

1

In [22]:
def lambda_break
  lambda {break 1; puts :unreachable}.call
  :reachable
end

lambda_break

:reachable

In [23]:
def wrap_method(proc_or_lambda)
  x = proc_or_lambda.call(1)
  x.to_s
end

wrap_method proc {|x| next x; x.succ}

"1"

In [24]:
wrap_method lambda {|x| next x; x.succ }

"1"

In [25]:
# 8-3-2 引数の違い

In [28]:
Proc.new {|x, y| x}.call(1)

1

In [27]:
lambda {|x, y| x}.call(1)

ArgumentError: wrong number of arguments (given 1, expected 2)

In [29]:
# 8-3-3 Proc#lambda?

In [31]:
args_proc = Proc.new {|x, y| [x, y]}
puts args_proc[1,2,3]
puts args_proc[1]
puts args_proc[[1,2]]

[1, 2]
[1, nil]
[1, 2]


In [32]:
args_lambda = lambda {|x, y| [x, y]}
puts args_lambda[1,2,3]

ArgumentError: wrong number of arguments (given 3, expected 2)

In [33]:
puts args_lambda[1]

ArgumentError: wrong number of arguments (given 1, expected 2)

In [34]:
puts args_lambda[[1,2]]

ArgumentError: wrong number of arguments (given 1, expected 2)

In [35]:
proc {}.lambda?

false

In [36]:
lambda {}.lambda?

true

In [37]:
1.method(:+).to_proc.lambda?

true

In [39]:
1.method(:+).to_proc.call()

ArgumentError: wrong number of arguments (given 0, expected 1)

In [40]:
1.method(:+).to_proc.call(2, 3)

ArgumentError: wrong number of arguments (given 2, expected 1)

In [41]:
# 8-4 Rubyでのクロージャ

In [42]:
# 8-4-1 クロージャとしてのProcオブジェクトの動作

In [43]:
def create_proc
  str = 'from create_proc'
  Proc.new {str}
end

:create_proc

In [45]:
proc_obj = create_proc

#<Proc:0x00005610346fa870@(pry):74>

In [46]:
str= 'from toplevel'
proc_obj.call

"from create_proc"

In [47]:
str = 'from toplevel'
top_level_proc = Proc.new {str}
top_level_proc.call

"from toplevel"

In [48]:
str += 'append strings'
top_level_proc.call

"from toplevelappend strings"

In [49]:
# 9章 Methodクラス

In [50]:
# 9-1 Methodオブジェクト

In [51]:
doubule = Proc.new {|x| x * 2}
doubule.(1)

2

In [52]:
array = [1,2,3,4,5]
array_shift = array.method(:shift)

#<Method: Array#shift>

In [53]:
array_shift.call

1

In [54]:
array

[2, 3, 4, 5]

In [56]:
class Dog
  def bark
    puts 'wan!!'
  end
end

dog = Dog.new
bark_wan = dog.method(:bark)

#<Method: Dog#bark>

In [57]:
class Dog
  def bark
    puts 'bowwow!!'
  end
end

:bark

In [59]:
dog.bark

bowwow!!


In [60]:
bark_wan.call

wan!!


In [61]:
def dog.bark
  puts 'kyankyan!!'
end

:bark

In [62]:
dog.bark

kyankyan!!


In [63]:
bark_wan.call

wan!!


In [64]:
# 9-1-2 メソッドに関する引数の情報

In [67]:
class Rec
  def rec(i)
    p i
    return if i.nil?
    self.rec(nil)
  end
end

Rec.new.rec(3)

3
nil


In [68]:
rec = Rec.new

#<Rec:0x00005610346a3408>

In [69]:
rec_method = rec.method(:rec)

#<Method: Rec#rec>

In [70]:
rec_method.call(3)

3
nil


In [75]:
class Rec
  def rec(i)
    p "override! #{i}"
    return if i.nil?
    rec(nil)
  end
end

:rec

In [77]:
rec_method.call(3) #再帰の場合はメソッドが書き換わってしまう！

3
"override! "


In [78]:
# 9-1-2 メソッドに関する引数の情報

In [85]:
class Arity
  def arity_0; end
  def arity_1(x); end
  def arity_2(x, y); end
  
  def arity_variable_length(*x); end
  def arity_variable_length_with_default_arg(x, *y); end
  def arity_default_val_1(x = 1); end
  def arity_default_val_2(x, y = 1); end
  
  def arity_keyword_2(x:, y:); end
end

:arity_keyword_2

In [87]:
arity_obj = Arity.new
puts arity_obj.method(:arity_0).arity
puts arity_obj.method(:arity_1).arity
puts arity_obj.method(:arity_2).arity
puts arity_obj.method(:arity_variable_length).arity
puts arity_obj.method(:arity_variable_length_with_default_arg).arity
puts arity_obj.method(:arity_default_val_1).arity
puts arity_obj.method(:arity_default_val_2).arity
puts arity_obj.method(:arity_keyword_2).arity

0
1
2
-1
-2
-1
-2
1


In [91]:
class MethodParameter
  def arity0; end
  def arity1(x); end
  def arity1_and_default_val(x = 1); end
  def arity1_and_valiable_arg(x, *y); end
  def arity1_and_block_arg(x, &y); end
end

:arity1_and_block_arg

In [94]:
method_params = MethodParameter.new
puts method_params.method(:arity0).parameters
puts method_params.method(:arity1).parameters
puts method_params.method(:arity1_and_default_val).parameters
puts method_params.method(:arity1_and_valiable_arg).parameters
puts method_params.method(:arity1_and_block_arg).parameters

[]
[[:req, :x]]
[[:opt, :x]]
[[:req, :x], [:rest, :y]]
[[:req, :x], [:block, :y]]


In [95]:
puts_method = Kernel.method(:puts)
puts_method.parameters

[[:rest]]

In [99]:
class KeywordArgs
  def normal_method_allow_keyword_args(first_arg, second_arg, third_arg)
    [first_arg, second_arg, third_arg]
  end
  
  private
  
  def method_missing(name, *args)
    method_name = name.to_s + '_allow_keyword_args'
    super unless respond_to? method_name
    args_hash = args.first
    
    new_args = self.method(method_name).parameters.map do |(req, arg_name)|
      args_hash[arg_name]
    end
    
    __send__ method_name, *new_args
  end
end

:method_missing

In [100]:
KeywordArgs.new.normal_method(third_arg: 1, first_arg: 2, second_arg: 3)

[2, 3, 1]

In [101]:
# 9-1-3 メソッドの持ち主/名前/レシーバ

In [104]:
class MethodInfo
  def sample_method; end
end

obj = MethodInfo.new
sample_method = obj.method(:sample_method)
sample_method.owner

MethodInfo

In [105]:
sample_method.name

:sample_method

In [107]:
sample_method.receiver

#<MethodInfo:0x0000561034d19c38>

In [108]:
sample_method.receiver == obj

true

In [109]:
#  9-1-4 メソッドがオーバーライドしたメソッド

In [115]:
class OriginalClass
  def hello
    'hello'
  end
end

class OverrideClass < OriginalClass
  def hello
    'hello hello'
  end
end

:hello

In [119]:
hello_method = OverrideClass.new.method(:hello)
puts hello_method
puts hello_method.owner
puts hello_method.call
puts hello_method.super_method
puts hello_method.super_method.owner
puts hello_method.super_method.call

#<Method: OverrideClass#hello>
OverrideClass
hello hello
#<Method: OriginalClass#hello>
OriginalClass
hello


In [121]:
p OriginalClass.new.method(:hello).super_method

nil
